In [2]:
import time

notebook_start_time = time.time()

In [3]:
import sys
from pathlib import Path

root_dir = str(Path().absolute().parent)
sys.path.append(root_dir)

In [46]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

from pprint import pprint
from loguru import logger

import polars as pl
import torch
from sentence_transformers import SentenceTransformer

from recsys.hopsworks_integration import feature_store
from recsys.config import settings
from recsys.raw_data_sources import load_datasets
from recsys.features.articles import compute_features_articles, generate_embeddings_for_dataframe
from recsys.features.customers import compute_features_customers, DatasetSampler
from recsys.features.transactions import compute_features_transactions
from recsys.features.interaction import generate_interaction_data
from recsys.features.ranking import compute_ranking_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Connect to Hopsworks Feature Store

In [5]:
project, fs = feature_store.get_feature_store()

2025-07-28 16:37:54.625 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:10 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-07-28 16:37:54,627 INFO: Initializing external client
2025-07-28 16:37:54,628 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-07-28 16:37:56,765 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/90261


## The H&M dataset

### Articles data

In [6]:
articles_df = load_datasets.extract_articles_df()
articles_df.shape

(105542, 25)

In [7]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"


In [8]:
articles_df.null_count()

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,416


#### feature engineering

In [9]:
articles_df = compute_features_articles(articles_df)
articles_df.shape

(105542, 27)

In [10]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""https://repo.hops.works/dev/jd…"
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""https://repo.hops.works/dev/jd…"
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""https://repo.hops.works/dev/jd…"


##### Create embeddings from the articles description

In [11]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2025-07-28 16:38:00.301 | INFO     | __main__:<module>:2 - Item 1:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Dark Black (Black)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-07-28 16:38:00.301 | INFO     | __main__:<module>:2 - Item 2:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Light White (White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-07-28 16:38:00.302 | INFO     | __main__:<module>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
Appearance: Stripe
Color: Dusty Light White (Off White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.


In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

2025-07-28 16:38:00.504 | INFO     | __main__:<module>:9 - Loading 'all-MiniLM-L6-v2' embedding model to device='cuda'


In [13]:
# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-07-28 16:38:00,563 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [14]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)

Generating embeddings: 100%|██████████| 105542/105542 [01:23<00:00, 1265.98it/s]


In [15]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.026782, 0.082344, … 0.022782]"
"""Strap top - Vest top in Garmen…","[-0.010396, 0.089874, … 0.022564]"
"""Strap top (1) - Vest top in Ga…","[-0.032753, 0.091124, … 0.022804]"


In [16]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10px; max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))

### Customers Data

In [17]:
customers_df = load_datasets.extract_customers_df()
customers_df.shape

(1371980, 7)

In [18]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [19]:
customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


##### feature engineering

In [20]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [21]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""


### Transactions Data

In [22]:
transactions_df =load_datasets.extract_transactions_df()
transactions_df.shape

(31788324, 5)

In [23]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id
date,str,i64,f64,i64
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2


#### feature engineering

In [24]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

In [25]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
0,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4
0,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4
0,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4


In [26]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)
dataset_subset = sampler.sample(
    customers_df=customers_df, transations_df=transactions_df
)
customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-07-28 16:40:30.170 | INFO     | recsys.features.customers:sample:77 - Sampling 1000 customers.
2025-07-28 16:40:30.294 | INFO     | recsys.features.customers:sample:80 - Number of transactions for all the customers: 31788324
2025-07-28 16:40:30.818 | INFO     | recsys.features.customers:sample:86 - Number of transactions for the 1000 sampled customers: 23799


In [27]:
transactions_df.shape

(23799, 9)

### Interaction data

In [28]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks: 100%|██████████| 1/1 [00:05<00:00,  5.47s/it]


(135916, 5)

In [29]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
-374400000,"""00b203a32faa3d007dba198ef27c15…","""749509002""",0,"""START"""
-363600000,"""00b203a32faa3d007dba198ef27c15…","""633162017""",0,"""749509002"""
-363600000,"""00b203a32faa3d007dba198ef27c15…","""820410001""",0,"""633162017"""
-363600000,"""00b203a32faa3d007dba198ef27c15…","""749509002""",0,"""820410001"""
-356400000,"""00b203a32faa3d007dba198ef27c15…","""820410001""",0,"""749509002"""


In [30]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73825
1,38292
2,23799


Here is what each score means:

    0 : No interaction between a customer and an item
    1 : A customer clicked an item
    2 : A customer bought an item

## Create Hopsworks Feature Groups

In [31]:
logger.info("Uploading 'customers' Feature Group to Hopsworks.")
customers_fg = feature_store.create_customers_feature_group(
    fs, df=customers_df, online_enabled=True
)

logger.info("✅ Uploaded 'customers' Feature Group to Hopsworks!")

2025-07-28 16:40:36.972 | INFO     | __main__:<module>:1 - Uploading 'customers' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 1000/1000 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: customers_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/customers_1_offline_fg_materialization/executions
2025-07-28 16:40:51,336 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-28 16:40:57,301 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-28 16:42:41,064 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-28 16:42:41,211 INFO: Waiting for log aggregation to finish.
2025-07-28 16:42:54,880 INFO: Execution finished successfully.


Online data ingestion progress: 0.00% |          | Rows 2000/?
2025-07-28 16:42:58.503 | INFO     | __main__:<module>:6 - ✅ Uploaded 'customers' Feature Group to Hopsworks!


In [32]:
logger.info("Uploading 'articles' Feature Group to Hopsworks.")
articles_fg = feature_store.create_articles_feature_group(
    fs,
    df=articles_df,
    articles_description_embedding_dim=model.get_sentence_embedding_dimension(),
    online_enabled=True,
)
logger.info("✅ Uploaded 'articles' Feature Group to Hopsworks!")

2025-07-28 16:42:58.572 | INFO     | __main__:<module>:1 - Uploading 'articles' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 105542/105542 | Elapsed Time: 05:08 | Remaining Time: 00:00


Launching job: articles_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/articles_1_offline_fg_materialization/executions
2025-07-28 16:48:20,084 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-07-28 16:48:23,490 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-28 16:48:26,171 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-28 16:51:24,984 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-28 16:51:25,115 INFO: Waiting for log aggregation to finish.
2025-07-28 16:51:48,075 INFO: Execution finished successfully.


Online data ingestion progress: 7.27% |▋         | Rows 7674/105542
2025-07-28 16:52:49.141 | INFO     | __main__:<module>:8 - ✅ Uploaded 'articles' Feature Group to Hopsworks!


In [33]:
logger.info("Uploading 'transactions' Feature Group to Hopsworks.")
trans_fg = feature_store.create_transactions_feature_group(
    fs=fs, df=transactions_df, online_enabled=True
)
logger.info("✅ Uploaded 'transactions' Feature Group to Hopsworks!")

2025-07-28 16:52:49.189 | INFO     | __main__:<module>:1 - Uploading 'transactions' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 23799/23799 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/transactions_1_offline_fg_materialization/executions
2025-07-28 16:53:04,559 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-28 16:53:07,729 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-28 16:54:56,958 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-28 16:54:57,090 INFO: Waiting for log aggregation to finish.
2025-07-28 16:55:22,894 INFO: Execution finished successfully.


Online data ingestion progress: 100.00% |██████████| Rows 23799/23799
2025-07-28 16:55:29.622 | INFO     | __main__:<module>:5 - ✅ Uploaded 'transactions' Feature Group to Hopsworks!


In [34]:
logger.info("Uploading 'interactions' Feature Group to Hopsworks.")
interactions_fg = feature_store.create_interactions_feature_group(
    fs=fs, df=interaction_df, online_enabled=True
)
logger.info("✅ Uploaded 'interactions' Feature Group to Hopsworks!!")

2025-07-28 16:55:29.665 | INFO     | __main__:<module>:1 - Uploading 'interactions' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 135916/135916 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/interactions_1_offline_fg_materialization/executions
2025-07-28 16:55:49,578 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-28 16:55:54,904 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-28 16:57:53,309 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-28 16:57:53,452 INFO: Waiting for log aggregation to finish.
2025-07-28 16:58:13,241 INFO: Execution finished successfully.


Online data ingestion progress: 100.00% |██████████| Rows 135916/135916
2025-07-28 16:58:16.152 | INFO     | __main__:<module>:5 - ✅ Uploaded 'interactions' Feature Group to Hopsworks!!


## Compute ranking dataset

In [47]:
ranking_df = compute_ranking_dataset(
    trans_fg,
    articles_fg,
    customers_fg,
)
ranking_df.shape

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.97s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.20s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (56.58s) 


(224136, 15)

In [48]:
ranking_df.head(3)

article_id,customer_id,age,label,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
str,str,f64,i32,str,str,str,str,str,str,str,str,str,str,str
"""839496003""","""34d30dcece38ac652e9fd05285d94d…",64.0,1,"""Top""","""Garment Upper body""","""All over pattern""","""Beige""","""Dark""","""Beige""","""Jersey fancy""","""Ladieswear""","""Ladieswear""","""Womens Everyday Collection""","""Jersey Fancy"""
"""777093001""","""91a30617b9a5b5ff3927779204a176…",49.0,1,"""Jumpsuit/Playsuit""","""Garment Full body""","""All over pattern""","""Dark Blue""","""Dark""","""Blue""","""Young Girl Dresses""","""Children Sizes 134-170""","""Baby/Children""","""Young Girl""","""Dresses/Skirts girls"""
"""875719003""","""23eeb5e9595c9409031f21a9c01fa3…",25.0,1,"""Trousers""","""Garment Lower body""","""Solid""","""Beige""","""Dusty Light""","""Mole""","""Trousers DS""","""Divided""","""Divided""","""Divided Selected""","""Trousers"""


In [49]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
0,203760
1,20376


In [50]:
logger.info("Uploading 'ranking' Feature Group to Hopsworks.")
rank_fg = feature_store.create_ranking_feature_group(
    fs,
    df=ranking_df,
    parents=[articles_fg, customers_fg, trans_fg],
    online_enabled=False,
)
logger.info("✅ Uploaded 'ranking' Feature Group to Hopsworks!!")

2025-07-28 17:11:32.493 | INFO     | __main__:<module>:1 - Uploading 'ranking' Feature Group to Hopsworks.


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/90261/fs/90180/fg/1495752


Uploading Dataframe: 100.00% |██████████| Rows 224136/224136 | Elapsed Time: 00:28 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/ranking_1_offline_fg_materialization/executions
2025-07-28 17:12:14,439 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-07-28 17:12:17,099 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-28 17:12:20,272 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-28 17:14:32,025 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-07-28 17:14:35,188 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-28 17:14:35,333 INFO: Waiting for log aggregation to finish.
2025-07-28 17:14:55,297 INFO: Execution finished successfully.


2025-07-28 17:15:04.101 | INFO     | __main__:<module>:8 - ✅ Uploaded 'ranking' Feature Group to Hopsworks!!


In [51]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2025-07-28 17:15:04.147 | INFO     | __main__:<module>:4 - ⌛️ Notebook Execution time: 2271.39 seconds ~ 37.86 minutes
